In [54]:
import os
import pandas as pd
import numpy as np

In [55]:
OBIPath = '/media/ak/Data/InterestRateFuturesData/ReconstructedLOB/OrderBookImbalance/'
OBIMFDFAResults = '/media/ak/Data/InterestRateFuturesData/ReconstructedLOB/OrderBookImbalance/OBIMFDFAResults'

In [86]:
resultsPath = os.path.join(OBIMFDFAResults, 'TwoSampleDataFrames')

In [56]:
def process_dict(input_dict):
    processed_dict = {}
    for key, value in input_dict.items():
        # Check if the value is a list of dictionaries
        if isinstance(value, list) and all(isinstance(item, dict) for item in value):
            for i, sub_dict in enumerate(value):
                for sub_key, sub_value in sub_dict.items():
                    new_key = f"{key}_{i}_{sub_key}"
                    # Process the sub_value similar to how other values are processed
                    if isinstance(sub_value, list) or (type(sub_value).__module__ == 'numpy' and isinstance(sub_value, np.ndarray)):
                        if len(sub_value) > 0:
                            try:
                                processed_dict[new_key] = [np.median(sub_value)]
                            except TypeError:
                                processed_dict[new_key] = sub_value
                        else:
                            processed_dict[new_key] = sub_value
                    else:
                        processed_dict[new_key] = [sub_value]
        elif key in ['list_gwidth', 'list_kernels', 'list_permuted_mmd2']:
            processed_dict[key] = value
        elif isinstance(value, list) or (type(value).__module__ == 'numpy' and isinstance(value, np.ndarray)):
            if len(value) > 0:
                try:
                    processed_dict[key] = [np.median(value)]
                except TypeError:
                    processed_dict[key] = value
            else:
                processed_dict[key] = value
        else:
            processed_dict[key] = [value]

    return processed_dict


In [57]:
def flatten_dict(d):
    def expand(key, value):
        if isinstance(value, dict):
            return [(str(key) + '_' + str(k), v) for k, v in flatten_dict(value).items()]
        elif isinstance(value, list) or (type(value).__module__ == 'numpy' and isinstance(value, np.ndarray)):
            if len(value) > 0:
                try:
                    return [(str(key), np.median(value))]
                except TypeError:
                    return [(str(key), value)]
            else:
                return [(str(key), value)]
        else:
            return [(str(key), value)]
    
    items = [item for k, v in d.items() for item in expand(k, v)]
    
    return dict(items)

# flattened_dict = flatten_dict(input_dict)


In [88]:
import pandas as pd

def process_and_flatten(file_path):
    """
    Process and flatten the contents of a pickle file.

    This function reads a pickle file specified by file_path, extracts
    and processes its contents (assuming a specific structure of the
    data), then flattens the dictionary structure into a single level. 
    Certain keys are removed from the final flattened dictionary as 
    part of data cleaning.

    Parameters:
    file_path (str): The path to the pickle file to be processed.

    Returns:
    dict: A flattened dictionary with specified keys removed.

    Note:
    - The function assumes the existence of 'process_dict' and 
      'flatten_dict' functions which are not defined in this snippet.
    - The structure of the pickle file and the dictionary keys are 
      assumed based on the function's implementation.
    """

    # Load the pickle file
    unPickledFile = pd.read_pickle(file_path)

    # Extract the first item's value from the dictionary
    resultsDict = unPickledFile[next(iter(unPickledFile))]

    # Process the dictionary (process_dict needs to be defined elsewhere)
    process_dict(resultsDict)

    # Flatten the dictionary (flatten_dict needs to be defined elsewhere)
    flattened_dict = flatten_dict(resultsDict)
    
    # List of keys to be removed from the flattened dictionary
    keys_to_remove = ['list_gwidth', 'list_kernels', 'best_ker']

    # Remove specified keys
    for key in keys_to_remove:
        flattened_dict.pop(key, None)
    
    return flattened_dict


In [62]:
symbolIDX = 3
symbols = os.listdir(OBIMFDFAResults)
symbolPath = os.path.join(OBIMFDFAResults, symbols[symbolIDX])
os.listdir(symbolPath)
keys = ['alpha', 'Spectrum', 'tau', 'Hlist']

In [120]:
keyIDX = 3
symbolKeyPath =  os.path.join(symbolPath, keys[keyIDX])
files = os.listdir(symbolKeyPath)
files

['YM1_list_H_shift_1_wind_10_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_1_wind_1_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_1_wind_4_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_1_wind_5_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_1_wind_6_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_1_wind_8_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_2_wind_10_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_2_wind_1_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_2_wind_4_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_2_wind_5_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_2_wind_6_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_2_wind_8_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_3_wind_10_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_3_wind_1_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_3_wind_4_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_3_wind_5_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_3_wind_6_OBI_quad_MMD_test.pkl',
 'YM1_list_H_shift_3_wind_8_OBI_quad_MMD_test.pkl']

In [72]:
# fileIdxPath 

In [73]:
# print([f for f in enumerate(resultsKeys) ])

In [76]:

# fileIdx = 1
for fileIdx, _ in enumerate(files):
    fileIdxPath = os.path.join(symbolKeyPath, files[fileIdx])
    shift_str, window_str = files[fileIdx].split("_shift_")[1].split("_OBI")[0].split("_wind_")
    # Convert the extracted strings to integers (or floats if needed)
    shift = int(shift_str)  # Use float(shift_str) if the value can be a decimal
    window = int(window_str)  # Use float(window_str) if the value can be a decimal

    #####################################################
    unPickledFile = pd.read_pickle(fileIdxPath)
    idXkeys = list(unPickledFile.keys())
    resultsDict = unPickledFile[idXkeys[0]]
    resultsKeys = list(resultsDict.keys())
    process_dict(resultsDict)
    flattened_dict = flatten_dict(resultsDict)
    # List of keys to remove
    keys_to_remove = ['list_gwidth', 'list_kernels', 'best_ker']

    # Remove the specified keys from the flattened dictionary
    for key in keys_to_remove:
        flattened_dict.pop(key, None)  # Use pop to avoid KeyError if key is not present

    # Now you can convert this flattened dictionary into a DataFrame
    df = pd.DataFrame([flattened_dict])
    df['window']= window
    df['shift']= shift
    print(df)

   perm_mmds1  chi2_weights  sim_mmds      sig2       Kxy      mean  \
0   -0.002899  1.029333e-19 -0.001549  0.057387  0.940443  0.009742   

            var       Kxx       Kyy  mean_gram  ...  XZ_test_test_stat  \
0  9.327089e-07  0.958063  0.950146   0.009742  ...           0.149949   

   XZ_test_h0_rejected  XZ_test_list_permuted_mmd2  YZ_test_alpha  \
0                 True                    -0.00274           0.05   

   YZ_test_pvalue  YZ_test_test_stat  YZ_test_h0_rejected  \
0          0.0295           0.026524                 True   

   YZ_test_list_permuted_mmd2  window  shift  
0                   -0.002423      10      1  

[1 rows x 26 columns]
   perm_mmds1  chi2_weights  sim_mmds      sig2       Kxy      mean  \
0   -0.002899  1.029333e-19 -0.001549  0.057387  0.940443  0.009742   

            var       Kxx       Kyy  mean_gram  ...  XZ_test_test_stat  \
0  9.327089e-07  0.958063  0.950146   0.009742  ...           0.045665   

   XZ_test_h0_rejected  XZ_test_list_

   perm_mmds1  chi2_weights  sim_mmds      sig2       Kxy      mean       var  \
0    -0.00223  6.942379e-19 -0.001453  0.297469  0.663572  0.124798  0.000211   

        Kxx       Kyy  mean_gram  ...  XZ_test_test_stat  XZ_test_h0_rejected  \
0  0.965897  0.505032   0.124798  ...           0.070009                 True   

   XZ_test_list_permuted_mmd2  YZ_test_alpha  YZ_test_pvalue  \
0                   -0.001972           0.05             0.0   

   YZ_test_test_stat  YZ_test_h0_rejected  YZ_test_list_permuted_mmd2  window  \
0           0.371239                 True                   -0.002897       1   

   shift  
0      3  

[1 rows x 26 columns]
   perm_mmds1  chi2_weights  sim_mmds      sig2       Kxy      mean       var  \
0    -0.00223  6.942379e-19 -0.001453  0.297469  0.663572  0.124798  0.000211   

        Kxx       Kyy  mean_gram  ...  XZ_test_test_stat  XZ_test_h0_rejected  \
0  0.965897  0.505032   0.124798  ...           0.110545                 True   

   XZ_test_

In [122]:

symbols = os.listdir(OBIMFDFAResults)
for symbolIDX in range(0,13):
    symbolPath = os.path.join(OBIMFDFAResults, symbols[symbolIDX])
    os.listdir(symbolPath)
    keys = ['alpha', 'Spectrum', 'tau', 'Hlist']
    keyIDX = 3

    symbolKeyPath =  os.path.join(symbolPath, keys[keyIDX])
    files = os.listdir(symbolKeyPath)
    # Using list comprehension to process each file and store the flattened dict
    dfs = [pd.DataFrame([process_and_flatten(os.path.join(symbolKeyPath, file))])
           .assign(**dict(zip(['window', 'shift'], map(int, file.split("_shift_")[1].split("_OBI")[0].split("_wind_")))))
           for file in files]

    # Concatenate all dataframes
    final_df = pd.concat(dfs, ignore_index=True)

    # final_df now contains all the data stacked together
    # Assuming symbols, symbolIDX, keys, and keyIDX are already defined in your code
    fileName = os.path.join(resultsPath,"_".join((symbols[symbolIDX], keys[keyIDX], "twoSampleResults.pkl")))

    # Save the DataFrame to a pickle file
    final_df.to_pickle(fileName)


FileNotFoundError: [Errno 2] No such file or directory: '/media/ak/Data/InterestRateFuturesData/ReconstructedLOB/OrderBookImbalance/OBIMFDFAResults/KE1/Hlist'

In [138]:

# Assuming final_df is your DataFrame

# Step 1: Calculate median for numeric columns
median_values = final_df.median()

# Step 2: Calculate proportion of 'True' for specified columns
def true_proportion(column):
    true_count = final_df[column].value_counts().get('True', 0)
    total_count = final_df[column].count()
    return true_count / total_count if total_count else 0

true_proportion_XZ = true_proportion('XZ_test_h0_rejected')
true_proportion_YZ = true_proportion('YZ_test_h0_rejected')

# Adding these proportions to the median_values Series for completeness
median_values['XZ_test_h0_rejected'] = true_proportion_XZ
median_values['YZ_test_h0_rejected'] = true_proportion_YZ

# median_values now contains the medians of numeric columns and the proportions of 'True' for the specified columns


In [139]:
median_values

perm_mmds1                   -2.465345e-03
chi2_weights                  3.647131e-19
sim_mmds                     -1.417493e-03
sig2                          8.869560e-02
Kxy                           9.139277e-01
mean                          1.354755e-02
var                           7.059816e-05
Kxx                           9.362612e-01
Kyy                           9.231269e-01
mean_gram                     1.354755e-02
var_gram                      7.059816e-05
med                           8.869560e-02
besti                         0.000000e+00
powers                        2.815154e+00
XZ_test_alpha                 5.000000e-02
XZ_test_pvalue                0.000000e+00
XZ_test_test_stat             1.651849e-01
XZ_test_h0_rejected           0.000000e+00
XZ_test_list_permuted_mmd2   -2.204394e-03
YZ_test_alpha                 5.000000e-02
YZ_test_pvalue                0.000000e+00
YZ_test_test_stat             2.871748e-01
YZ_test_h0_rejected           0.000000e+00
YZ_test_lis

In [83]:
fileName = "_".join((symbols[symbolIDX], keys[keyIDX], "twoSampleResults.pkl"))

In [147]:
final_df['XZ_test_h0_rejected'].value_counts()

True     17
False     1
Name: XZ_test_h0_rejected, dtype: int64

In [150]:
def proportion_second_value(column):
    total_count = final_df[column].count()
    if total_count > 0 and len(final_df[column].value_counts()) > 1:
        second_value_count = final_df[column].value_counts()[1]
        proportion = (second_value_count / total_count) * 100
    else:
        proportion = 0
    return f"{proportion:.2f}%"

# Calculate the proportion of the second value (which is 'True' in your case)
true_proportion_XZ = proportion_second_value('XZ_test_h0_rejected')
true_proportion_YZ = proportion_second_value('YZ_test_h0_rejected')

# If you need to round numeric columns to two decimals
median_values = final_df.median().round(3)

# Store the formatted proportions in the median_values Series
median_values['XZ_test_h0_rejected'] = true_proportion_XZ
median_values['YZ_test_h0_rejected'] = true_proportion_YZ


In [151]:
median_values

perm_mmds1                    -0.002
chi2_weights                     0.0
sim_mmds                      -0.001
sig2                           0.089
Kxy                            0.914
mean                           0.014
var                              0.0
Kxx                            0.936
Kyy                            0.923
mean_gram                      0.014
var_gram                         0.0
med                            0.089
besti                            0.0
powers                         2.815
XZ_test_alpha                   0.05
XZ_test_pvalue                   0.0
XZ_test_test_stat              0.165
XZ_test_h0_rejected           94.44%
XZ_test_list_permuted_mmd2    -0.002
YZ_test_alpha                   0.05
YZ_test_pvalue                   0.0
YZ_test_test_stat              0.287
YZ_test_h0_rejected           83.33%
YZ_test_list_permuted_mmd2    -0.002
window                           2.0
shift                            5.5
dtype: object